In [63]:
# Cuts used to go from summary.fits to cut.fits, for now we will use the cut.fits file

#snr = 5.0
#sharp = 0.04
#crowd = 0.5
#objtype = 1
#flag = 99
#importing modules

from astropy.io import fits
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.coordinates import match_coordinates_sky
import numpy as np
import os


In [75]:
#DECLARING DIRECTORIES
#path to folder of summary/cut.fits with format sn_name-filter-mjd_#-cut/sum.fits i.e: 2021gmj-F625W-mjd_59574-cut.fits
#ONLY FITS FILES IN FOLDER!!!!
path = '/Users/Dominic/Lamat/2014C/photometry/summary/'
#where full region files will be written
outpath_full = '/Users/Dominic/Lamat/2014C/photometry/final_catalogs_full/'
outpath_cut = '/Users/Dominic/Lamat/2014C/photometry/final_catalogs_cut/'
#where cut region files will be written
#position in ra and dec (degrees)
ra = 339.27333
dec = 34.40886
#search radius (arc seconds)
radius = 1

In [65]:
#writes region files to outpath
#the centre of ds9 pixel is (1,1), but centre of dolphot pixel is (0.5,0.5)
#to make it in ds9 format it’s (+0.5,+0.5) to dolphot coordinates
def writeregionfile(filename, outpath, objlist, color="blue",sys=''):
    if sys == '': 
        sys = 'wcs'
    completeName = os.path.join(outpath, filename)
    out = open(completeName, 'w')
    i = -1
    out.write('# Region file format: DS9 version 4.0\nglobal color='+color+' font="helvetica 10 normal" select=1 highlite=1 edit=1 move=1 delete=1 include=1 fixed=0 source\n')
    if sys == 'wcs':
      out.write('fk5\n')
      for i in range(len(objlist)):
        out.write("point(%.7f,%.7f) # point=boxcircle text={%i}\n" % (objlist.RA[i],objlist.DEC[i], i))
    if sys == 'img':
      out.write('image\n')
      for i in range(len(objlist)):
        out.write("point(%.3f,%.3f) # point=boxcircle text={%i}\n" % (objlist.X[i] + 0.5 ,objlist.Y[i] + 0.5, i))
    out.close()
    print("writing region file:", filename)


In [66]:
#finds sn name, filter, mjd (date), and search radius
def find_attributes(filename, radius):
    global sn_name
    global filt
    global mjd
    global search_radius
    #reset attributes
    sn_name = ''
    filt = ''
    mjd = ''
    search_radius = 0
    #split name
    filename_split = filename.split("-")#split filename into attributes
    
    #find name
    sn_name = filename_split[0]
    print("SN name: " + sn_name)
    #find filt
    filt = filename_split[1]
    print("filt: " + filt)
    #find mjd    
    mjd = filename_split[2]
    print("MJD: " + mjd)
    
    search_radius = radius * u.arcsec
    print('Search radius: ' + str(search_radius))

In [68]:
#creates cut region files using cut.fits
#rai, deci is ra and dec of position, radius is search radius
def returnregionfilescut(path, rai, deci, radius, outpath): #use this for cut.fits
    dir_list = os.listdir(path)
    #for each file in path
    for file in os.listdir(path):
        if file != '.DS_Store': #weird mac meta file
            filename = os.fsdecode(file)
            print(filename)
            find_attributes(filename, radius)
            sn_position = SkyCoord(rai*u.degree, +deci*u.degree)
            cut_catalog=fits.open(path+filename, ignore_missing_simple=True)[1]
            data_cut=cut_catalog.data
            tbl_cut=Table(data_cut)
            radec_cut = SkyCoord(ra=data_cut.RA*u.degree, dec=data_cut.DEC*u.degree)
            d2d_cut = sn_position.separation(radec_cut)
            index_cut= d2d_cut < search_radius
            data_searched_cut=data_cut[index_cut]
            tbl_searched_cut=tbl_cut[index_cut]
            writeregionfile(sn_name+"-"+filt+"-"+mjd+"-"+str(search_radius.value) +"_arcsec-ra_dec_cut.reg", outpath, data_searched_cut, "red", "wcs")
            writeregionfile(sn_name+"-"+filt+"-"+mjd+"-"+str(search_radius.value) +"_arcsec-pixel_xy_cut.reg", outpath, data_searched_cut, "red", "img")

In [69]:
#creates full region files using summary.fits
#rai, deci is ra and dec of position, radius is search radius
def returnregionfilesfull(path, rai, deci, radius, outpath): #use this for summary.fits
    dir_list = os.listdir(path)
    #for each file in path
    for file in os.listdir(path):
        if file != '.DS_Store': #weird mac meta file
            filename = os.fsdecode(file)
            print(filename)
            find_attributes(filename, radius) #get name, mjd, filt
            sn_position = SkyCoord(rai*u.degree, +deci*u.degree)
            full_catalog=fits.open(path+filename, ignore_missing_simple=True)[1]
            data_full=full_catalog.data
            tbl_full=Table(data_full)
            radec_full = SkyCoord(ra=data_full.RA*u.degree, dec=data_full.DEC*u.degree)
            d2d_full = sn_position.separation(radec_full)
            index_full= d2d_full < search_radius
            data_searched_full=data_full[index_full]
            tbl_searched_full=tbl_full[index_full]
            writeregionfile(sn_name+"-"+filt+"-"+mjd+"-"+str(search_radius.value) +"_arcsec-ra_dec_full.reg", outpath, data_searched_full, "red", "wcs")
            writeregionfile(sn_name+"-"+filt+"-"+mjd+"-"+str(search_radius.value) +"_arcsec-pixel_xy_full.reg", outpath, data_searched_full, "red", "img")

In [78]:
#returnregionfiles(path, ra, dec, search radius)
#returning region files of summary.fits
#path should point to folder of summary.fits in format previously stated
#SN2021do_summary
returnregionfilesfull(path, ra, dec, radius, outpath_full)

2014C-F555W-mjd_59792-summary.fits
SN name: 2014C
filt: F555W
MJD: mjd_59792
Search radius: 1.0 arcsec
writing region file: 2014C-F555W-mjd_59792-1.0_arcsec-ra_dec_full.reg
writing region file: 2014C-F555W-mjd_59792-1.0_arcsec-pixel_xy_full.reg
2014C-F814W-mjd_59792-summary.fits
SN name: 2014C
filt: F814W
MJD: mjd_59792
Search radius: 1.0 arcsec
writing region file: 2014C-F814W-mjd_59792-1.0_arcsec-ra_dec_full.reg
writing region file: 2014C-F814W-mjd_59792-1.0_arcsec-pixel_xy_full.reg


In [79]:
#returnregionfiles(path, ra, dec, search radius)
#returning region files of cut.fits
#path should point to folder of cut.fits in format previously stated
#SN2021do_cut
returnregionfilescut(path, ra, dec, radius, outpath_cut)

2014C-F555W-mjd_59792-summary.fits
SN name: 2014C
filt: F555W
MJD: mjd_59792
Search radius: 1.0 arcsec
writing region file: 2014C-F555W-mjd_59792-1.0_arcsec-ra_dec_cut.reg
writing region file: 2014C-F555W-mjd_59792-1.0_arcsec-pixel_xy_cut.reg
2014C-F814W-mjd_59792-summary.fits
SN name: 2014C
filt: F814W
MJD: mjd_59792
Search radius: 1.0 arcsec
writing region file: 2014C-F814W-mjd_59792-1.0_arcsec-ra_dec_cut.reg
writing region file: 2014C-F814W-mjd_59792-1.0_arcsec-pixel_xy_cut.reg
